<a href="https://colab.research.google.com/github/paulhuangkm/MiniASR/blob/main/example/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MiniASR Tutorial: LibriSpeech Training**
This is a tutorial for training an end-to-end automatic speech recognition model with the toolkit [MiniASR](https://github.com/vectominist/MiniASR).  
You can run this notebook on [Google Colab](colab.research.google.com/), but to train an ASR model completely requires a Pro account since it needs several hours to converge.

## **Download Code & Install Dependencies**
Ref: [MiniASR](https://github.com/vectominist/MiniASR)

In [2]:
! git clone https://github.com/paulhuangkm/MiniASR.git
% cd MiniASR

Cloning into 'MiniASR'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 417 (delta 0), reused 1 (delta 0), pack-reused 414
Receiving objects: 100% (417/417), 333.62 KiB | 2.95 MiB/s, done.
Resolving deltas: 100% (220/220), done.


UsageError: Line magic function `%` not found.


In [3]:
! pip3 install -e ./

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/jonathanhsu/MiniASR/example/data
ERROR: file:///home/jonathanhsu/MiniASR/example/data does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


## **Download Data**
- training set: [Libri-light](https://github.com/facebookresearch/libri-light) fine-tuning set (10 hours, 0.6G)
- development set: [LibriSpeech](https://www.openslr.org/12) `dev-clean` set
- testing set: [LibriSpeech](https://www.openslr.org/12) `test-clean` set

In [1]:
! mkdir -p data
%cd data
! wget https://dl.fbaipublicfiles.com/librilight/data/librispeech_finetuning.tgz
! tar zxf librispeech_finetuning.tgz
! rm librispeech_finetuning.tgz

/home/jonathanhsu/MiniASR/example/data
--2022-02-28 16:04:23--  https://dl.fbaipublicfiles.com/librilight/data/librispeech_finetuning.tgz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 597601132 (570M) [application/gzip]
Saving to: ‘librispeech_finetuning.tgz’

librispeech_finetun 100%[===================>] 569.92M  21.2MB/s    in 28s     

2022-02-28 16:04:52 (20.1 MB/s) - ‘librispeech_finetuning.tgz’ saved [597601132/597601132]



In [4]:
! wget https://www.openslr.org/resources/12/dev-clean.tar.gz
! wget https://www.openslr.org/resources/12/test-clean.tar.gz
! tar zxf dev-clean.tar.gz
! tar zxf test-clean.tar.gz
! rm dev-clean.tar.gz
! rm test-clean.tar.gz
%cd ..

--2022-02-28 16:06:38--  https://www.openslr.org/resources/12/dev-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337926286 (322M) [application/x-gzip]
Saving to: ‘dev-clean.tar.gz’

dev-clean.tar.gz    100%[===================>] 322.27M   686KB/s    in 6m 31s  

2022-02-28 16:13:11 (844 KB/s) - ‘dev-clean.tar.gz’ saved [337926286/337926286]

--2022-02-28 16:13:11--  https://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘test-clean.tar.gz’

test-clean.tar.gz   100%[===================>] 330.60M  1.90MB/s    in 5m 32s  

2022-02-28 16:18:44 (1020 KB/s) - ‘test-clean.tar.gz’ saved [3466

## **Preprocess Data**
Find all data in the corpus and extract vocabularies. We use characters as text tokens since the dataset is small.

In [5]:
# Train set
! miniasr-preprocess \
        -c LibriSpeech \
        -p data/librispeech_finetuning \
        -s 1h \
        -o data/libri_train_1h \
        --gen-vocab \
        --char-vocab-size 40

! miniasr-preprocess \
        -c LibriSpeech \
        -p data/librispeech_finetuning \
        -s 9h \
        -o data/libri_train_9h

# Development set
! miniasr-preprocess \
        -c LibriSpeech \
        -p data/LibriSpeech \
        -s dev-clean \
        -o data/libri_dev

# Test set
! miniasr-preprocess \
        -c LibriSpeech \
        -p data/LibriSpeech \
        -s test-clean \
        -o data/libri_test

02-28 16:19 run_preprocess.py.main(72) Preprocessing LibriSpeech corpus.
02-28 16:19 run_preprocess.py.main(73) Subsets = ['1h']
02-28 16:19 run_preprocess.py.main(76) Results will be saved to data/libri_train_1h
02-28 16:19 run_preprocess.py.main(79) Reading data from data/librispeech_finetuning
02-28 16:19 run_preprocess.py.main(85) Found 286 audio files.
02-28 16:19 run_preprocess.py.main(89) Saving unsorted data dict to data/libri_train_1h/data_dict.json
02-28 16:19 run_preprocess.py.main(94) Sorting data by audio file length.
02-28 16:19 run_preprocess.py.main(103) Saving sorted data list to data/libri_train_1h/data_list_sorted.json
02-28 16:19 run_preprocess.py.main(109) Generating LM file.
02-28 16:19 run_preprocess.py.main(117) Generating vocabularies.
02-28 16:19 run_preprocess.py.main(120) Generating characters.
Found 28 chars.
Selected 28 vocabularies.
Saving char vocabularies to data/libri_train_1h/vocab_char.txt
02-28 16:19 run_preprocess.py.main(72) Preprocessing LibriSpe

## **Training**
- Modify `MiniASR/egs/librispeech/config/ctc_train_example.yaml` for changing training hyper-parameters.
- The results will be saved to `MiniASR/model/ctc_libri-10h_char`.

In [6]:
! mkdir -p model

In [7]:
! pwd

/home/jonathanhsu/MiniASR/example


In [14]:
! minasr-asr --config ../egs/librispeech/config/con_ctc.yaml

# Resume training with this command:
# ! minasr-asr --ckpt model/con_libri-10h_char/epoch=4-step=429.ckpt

02-28 16:28 run_asr.py.main(99) Training mode.
02-28 16:28 dataloader.py.create_dataloader(86) Creating text tokenizer of character level.
02-28 16:28 dataloader.py.create_dataloader(91) Generating datasets and dataloaders. (mode = train)
02-28 16:28 dataset.py.__init__(28) Loading data from ['data/libri_train_1h/data_list_sorted.json', 'data/libri_train_9h/data_list_sorted.json']
100%|████████████████████████████████████| 2763/2763 [00:00<00:00, 31793.95it/s]
02-28 16:28 dataset.py.__init__(49) 2763 audio files found (mode = train)
02-28 16:28 dataset.py.__init__(28) Loading data from ['data/libri_dev/data_list_sorted.json']
100%|████████████████████████████████████| 2703/2703 [00:00<00:00, 52287.83it/s]
02-28 16:28 dataset.py.__init__(49) 2703 audio files found (mode = dev)
02-28 16:28 asr_trainer.py.create_asr_trainer(24) Creating ASR model (type = con_asr).
Downloading: "https://github.com/s3prl/s3prl/archive/master.zip" to /home/jonathanhsu/.cache/torch/hub/master.zip
[s3prl.upstr

## **Testing**
- Specify your checkpoint with `--ckpt`.

In [5]:
! minasr-asr \
    --config ../egs/librispeech/config/con_ctc_test.yaml \
    --test \
    --override "args.data.dev_paths=['data/libri_test/data_list_sorted.json']" \
    --ckpt model/con_libri-10h_char/epoch=174-step=15224.ckpt

02-28 18:36 basic_setups.py.override(89) Override: args.data.dev_paths = ['data/libri_test/data_list_sorted.json']
02-28 18:36 run_asr.py.main(105) Testing mode.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec2.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.hubert.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.distiller.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.roberta.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.vq_wav2vec.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.decoar2.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not 

## **Inference**

In [ ]:
from miniasr.utils import load_from_checkpoint, sequence_distance
from miniasr.data.audio import load_waveform

model, args, tokenizer = load_from_checkpoint(
    'model/ctc_libri-10h_char/epoch=44-step=3869.ckpt', 'cuda')
waves = [load_waveform('data/LibriSpeech/dev-clean/6345/93302/6345-93302-0025.flac').to('cuda')]
hyps = model.recognize(waves)

In [ ]:
print(hyps[0])
ref = 'ARE YOU REALLY GOING TO THROW ME OVER FOR A THING LIKE THIS'
res_cer = sequence_distance(ref, hyps[0], mode='char')
res_wer = sequence_distance(ref, hyps[0], mode='word')
print('CER = {:.2f}%'.format(100. * res_cer['distance'] / res_cer['length']))
print('WER = {:.2f}%'.format(100. * res_wer['distance'] / res_wer['length']))